In [19]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  

import datetime
import numpy
import modeltools.grid
import matplotlib.pyplot
import modeltools.hycom.io
import modeltools.hycom.blkdat
import modeltools.hycom.timetools
import modeltools.hycom.indata


import modeltools.cice.io
import modeltools.forcing.atmosphere
import modeltools.tools.interpolation
import pylab
import scipy.interpolate
import time
import random

# Define (or read) grid
grid1=modeltools.grid.Proj4Grid("+proj=stere  +lon_0=-45 +lat_0=90 +lat_ts=80 +ellps=sphere",
                          -89.5,45.5,20000,20000,400,300)
mlon,mlat=grid1.pgrid()

# parse blkdat to get yearflag
blkd = modeltools.hycom.blkdat.Parser("blkdat.input")
yrflag = blkd["yrflag"]
wndflg = blkd["wndflg"]
print "wndflg=",wndflg

# Forcing metadata from xml file
af=modeltools.forcing.atmosphere.AtmosphericForcing("era-interim.xml","era-interim")
known_names=af.known_names # "Known names" used by forcing module

# Open output files. Dict uses "known name" when mapping to file object
ffiles={}
for k,vname in modeltools.hycom.indata.atmosphere_variable_names.items() :
    if k in known_names :
        ffiles[k]=modeltools.hycom.io.ABFileForcing("forcing.%s"%vname,"w",idm=grid1.Nx, jdm=grid1.Ny,
                                                    cline1="ERA Interim",cline2=vname)



# Set time limits
start=datetime.datetime(1991,2,3,12,0,0)
end=datetime.datetime(1991,2,10,12,0,0)
timestep=af.timestep
timestepday=timestep.days+timestep.seconds/86400.


# Main loop 
dt = start
while dt <= end :
    
    print "Reading at ",dt

    # Convert datetime object to ordinal date, then to hycom "dtime"
    ord_day,hour=modeltools.hycom.timetools.datetime_to_ordinal(dt,yrflag)
    dtime=modeltools.hycom.timetools.dayfor(dt.year,ord_day,hour,yrflag)

    # Read variables
    tmpfld=af.get_timestep(dt,varnames=known_names)
    coords=af.get_coords(dt,varnames=known_names)
    
    #raise NameError,"test"
    
    # Estimate dependent variable on native grid
    if wndflg in [1,2,3] :
        af.calculate_windstress()
        af.calculate_windspeed()
        af.calculate_ustar()
    af.calculate_vapmix()

    
    
    # Loop over open files 
    for kn in [elem for elem in known_names if elem in ffiles.keys()] :
            
        # NAme used by hycom
        vname=modeltools.hycom.indata.atmosphere_variable_names[kn]

        # Read fields using known names
        lo,la=coords[kn]
        fld=tmpfld[kn]

        fi=modeltools.tools.interpolation.FieldInterpolatorBilinear(lo,la,fld,mlon,mlat)
        newfld=fi.interpolate(fld)
        #fi=modeltools.tools.interpolation.FieldInterpolatorRectBivariateSpline(lo,la,fld,mlon,mlat)
        #newfld=fi.interpolate(fld)

        # Write to forcing file
        ffiles[kn].writefield(newfld,newfld,vname,dtime,timestepday)
    
    # Increase time
    dt = dt + timestep
        
for kn in [elem for elem in known_names if elem in ffiles.keys()] :
    ffile[kn].close()
af=[]



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(302, 402)

DEBUG:root:Initialized P-grid using projection +proj=stere  +lon_0=-45 +lat_0=90 +lat_ts=80 +ellps=sphere
DEBUG:root:Lower left corner lon/lat of grid: (-89.5,45.5)
DEBUG:root:Grid spacing in projection coords: (2e+04,2e+04)
DEBUG:root:Number of grid Nodes in x/y      : (  400,  300)
DEBUG:root:Min   x projection coordinate = -3.65e+06
DEBUG:root:Max   x projection coordinate = 4.37e+06
DEBUG:root:Min   y projection coordinate = -3.71e+06
DEBUG:root:Max   y projection coordinate = 2.31e+06
DEBUG:root:Min lon = -180
DEBUG:root:Max lon = 180
DEBUG:root:Min lat = 41.2
DEBUG:root:Max lat = 89.9


 (402,) (302,)
wndflg=

DEBUG:root:Endianness set to big
DEBUG:root:Endianness set to big
DEBUG:root:Endianness set to big
DEBUG:root:Endianness set to big
DEBUG:root:Endianness set to big
DEBUG:root:Endianness set to big


 4
Reading at  1991-02-03 12:00:00
Unit conversion: SSRD unit= 21600**-1 W m**2 s**-1 targetunit= W m**2 s**-1
Unit conversion:max= 2.11855e+07
Unit conversion:max after= 980.812
Reading at  1991-02-03 18:00:00
Unit conversion: SSRD unit= 21600**-1 W m**2 s**-1 targetunit= W m**2 s**-1
Unit conversion:max= 2.2272e+07
Unit conversion:max after= 1031.11
Reading at  1991-02-04 00:00:00
Unit conversion: SSRD unit= 21600**-1 W m**2 s**-1 targetunit= W m**2 s**-1
Unit conversion:max= 2.11528e+07
Unit conversion:max after= 979.295
Reading at  1991-02-04 06:00:00
Unit conversion: SSRD unit= 21600**-1 W m**2 s**-1 targetunit= W m**2 s**-1
Unit conversion:max= 2.17723e+07
Unit conversion:max after= 1007.98
Reading at  1991-02-04 12:00:00
Unit conversion: SSRD unit= 21600**-1 W m**2 s**-1 targetunit= W m**2 s**-1
Unit conversion:max= 2.15562e+07
Unit conversion:max after= 997.973
Reading at  1991-02-04 18:00:00
Unit conversion: SSRD unit= 21600**-1 W m**2 s**-1 targetunit= W m**2 s**-1
Unit conve

NameError: name 'ffile' is not defined